<a href="https://colab.research.google.com/github/AnastasiiaVladimirovna/transcriptomics_homeworks/blob/main/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка нужных пакетов
!apt-get install libgmp3-dev libcairo2-dev libharfbuzz-dev libfribidi-dev libudunits2-dev
!gdown "1W0TuuQospxcasOX5cQdhT_Xd-k7-QAck&confirm=t"
!tar xf library.tar.gz
!pip3 install scanpy[leiden] anndata2ri

# Необходимо, чтобы конкретно прогрузился matplotlib
import os
os.kill(os.getpid(), 9)

In [1]:
import anndata2ri
anndata2ri.activate()

In [2]:
# Загрузка R magic
!pip install rpy2==3.5.1
%load_ext rpy2.ipython
%R .libPaths("usr/local/lib/R/site-library/")
# %load_ext rpy2.ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'/content/...,'/usr/loca...,'/usr/lib/...,'/usr/lib/...


In [7]:
!gdown "1Ctq4EmV2ps1dMiFcbqW467ywezRSGGN0&confirm=t"
!unzip PBMC_10k.zip

Downloading...
From: https://drive.google.com/uc?id=1Ctq4EmV2ps1dMiFcbqW467ywezRSGGN0&confirm=t
To: /content/PBMC_10k.zip
100% 345M/345M [00:02<00:00, 168MB/s]
Archive:  PBMC_10k.zip
replace filtered_feature_bc_matrix.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: filtered_feature_bc_matrix.h5  
  inflating: filtered_feature_bc_matrix.tar.gz  
  inflating: metrics_summary.csv     
  inflating: raw_feature_bc_matrix.h5  
  inflating: raw_feature_bc_matrix.tar.gz  
  inflating: web_summary.html        


In [8]:
!tar -xvzf filtered_feature_bc_matrix.tar.gz

filtered_feature_bc_matrix/
filtered_feature_bc_matrix/matrix.mtx.gz
filtered_feature_bc_matrix/features.tsv.gz
filtered_feature_bc_matrix/barcodes.tsv.gz


In [9]:
import pandas as pd
import numpy as np
import seaborn as sns

In [10]:
import scanpy as sc

adata = sc.Aread_10x_mtx("filtered_feature_bc_matrix") # Эта функция позволяет быстро прочитать аутпут программы Cell Ranger
print(adata)

AnnData object with n_obs × n_vars = 11996 × 36601
    var: 'gene_ids', 'feature_types'


# Номер 1

Итак, функция с таблицами. На вход получает саму Anndata, столбец, в котором написаны группы и название части Anndata, в которой есть этот столбец (чтобы понимать выкидывать строки или столбцы)
На выходе получается словарь, где таблицы хранятся под ключами вида **adata_'** *название группы в столбике*'

In [ ]:
def splitt(adata, a_part):
  d = {}
  u = list(set(list(a_part)))
  for i in u:
    ind = list([a_part == i][0])
    d['adata_' + i] = adata[ind, :]
  return d


Проверим, что работает

In [ ]:
adata.obs['batch'] = ['1'] * 10000 + ['2'] * 1995 + ['3']

In [ ]:
adata.obs

,batch
AAACCCAAGGCCCAAA-1,1
AAACCCAAGTAATACG-1,1
AAACCCAAGTCACACT-1,1
AAACCCACAAAGCGTG-1,1
AAACCCACAATCGAAA-1,1
...,...
TTTGTTGGTTGGATCT-1,2
TTTGTTGGTTTCTTAC-1,2
TTTGTTGTCCATTTCA-1,2
TTTGTTGTCTACACAG-1,2


In [ ]:
a_part = adata.obs['batch']
d = splitt(adata, a_part)

In [ ]:
print(d['adata_1'].X.A.shape, d['adata_2'].X.A.shape, d['adata_3'].X.A.shape)

(10000, 36601) (1995, 36601) (1, 36601)


# Номер 2

In [ ]:
import random
def random_adata(M, N, adata):
  shape = adata.X.A.shape
  colnum = shape[1]
  rownum = shape[0]
  randomcol = random.sample(range(0, colnum + 1), M)
  randomrow = random.sample(range(0, rownum + 1), N)
  adata_new = adata[randomrow, randomcol]
  return adata_new

In [ ]:
adata_new = random_adata(570, 570, adata)

In [ ]:
adata_new.shape

(570, 570)

# Номер 3

In [14]:
import rpy2.robjects as ro

In [ ]:
%%R
library(Seurat) 

In [11]:
def seurat_format(adata):
  ro.r('library(Seurat)')
  ro.globalenv['adata'] = adata
  adata_new = ro.r('adata.to.seurat <- as.Seurat(adata, data = NULL, counts = "X")')
  return adata_new

In [ ]:
adata_new = seurat_format(adata)